<center>
<img src="../../img/ods_stickers.jpg">
## Открытый курс по машинному обучению. Сессия № 2
Автор материала: программист-исследователь Mail.ru Group, старший преподаватель Факультета Компьютерных Наук ВШЭ Юрий Кашницкий. Материал распространяется на условиях лицензии [Creative Commons CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/). Можно использовать в любых целях (редактировать, поправлять и брать за основу), кроме коммерческих, но с обязательным упоминанием автора материала.

# <center>Домашнее задание № 10
## <center> Прогнозирование задержек вылетов

Ваша задача – побить как минимум 2 бенчмарка в [соревновании](https://www.kaggle.com/c/flight-delays-2017) на Kaggle Inclass. Подробных инструкций не будет, будет только тезисно описано, как получен второй – с помощью Xgboost. Надеюсь, на данном этапе курса вам достаточно бросить полтора взгляда на данные, чтоб понять, что это тот тип задачи, в которой затащит градиентный бустинг. Скорее всего Xgboost, но тут у нас еще немало категориальных признаков...

<img src='../../img/xgboost_meme.jpg' width=40% />

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score

C:\Users\Qoffee\Anaconda3v2\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
train = pd.read_csv('../../data/flight_delays_train.csv')
test = pd.read_csv('../../data/flight_delays_test.csv')

In [3]:
train.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance,dep_delayed_15min
0,c-8,c-21,c-7,1934,AA,ATL,DFW,732,N
1,c-4,c-20,c-3,1548,US,PIT,MCO,834,N
2,c-9,c-2,c-5,1422,XE,RDU,CLE,416,N
3,c-11,c-25,c-6,1015,OO,DEN,MEM,872,N
4,c-10,c-7,c-6,1828,WN,MDW,OMA,423,Y


In [4]:
test.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance
0,c-7,c-25,c-3,615,YV,MRY,PHX,598
1,c-4,c-17,c-2,739,WN,LAS,HOU,1235
2,c-12,c-2,c-7,651,MQ,GSP,ORD,577
3,c-3,c-25,c-7,1614,WN,BWI,MHT,377
4,c-6,c-6,c-3,1505,UA,ORD,STL,258


Итак, надо по времени вылета самолета, коду авиакомпании-перевозчика, месту вылета и прилета и расстоянию между аэропортами вылета и прилета предсказать задержку вылета более 15 минут. В качестве простейшего бенчмарка возьмем логистическую регрессию и два признака, которые проще всего взять: `DepTime` и `Distance`. У такой модели результат – 0.68202 на LB. 

In [5]:
X_train, y_train = train[['Distance', 'DepTime']].values, train['dep_delayed_15min'].map({'Y': 1, 'N': 0}).values
X_test = test[['Distance', 'DepTime']].values

X_train_part, X_valid, y_train_part, y_valid = train_test_split(X_train, y_train, test_size=0.3, random_state=17)

scaler = StandardScaler()
X_train_part = scaler.fit_transform(X_train_part)
X_valid = scaler.transform(X_valid)

C:\Users\Qoffee\Anaconda3v2\lib\site-packages\sklearn\utils\validation.py:444: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [6]:
X_train, y_train = train[['Month', 'DayofMonth', 'DayOfWeek', 'DepTime', 'UniqueCarrier',
       'Origin', 'Dest', 'Distance']], train['dep_delayed_15min'].map({'Y': 1, 'N': 0})
X_test = test[['Month', 'DayofMonth', 'DayOfWeek', 'DepTime', 'UniqueCarrier',
       'Origin', 'Dest', 'Distance']]
frames = [X_train, X_test]
X = pd.concat(frames)
cat_features = X.columns[X.dtypes == 'object']
df_interact = X.copy()
for i, col1 in enumerate(cat_features):
    for j, col2 in enumerate(cat_features[i + 1:]):
        df_interact[col1 + '_' + col2] = df_interact[col1] + '_' + df_interact[col2]

In [ ]:
X_train, y_train = train[['Month', 'DayofMonth', 'DayOfWeek', 'DepTime', 'UniqueCarrier',
       'Origin', 'Dest', 'Distance']], train['dep_delayed_15min'].map({'Y': 1, 'N': 0})
X_test = test[['Month', 'DayofMonth', 'DayOfWeek', 'DepTime', 'UniqueCarrier',
       'Origin', 'Dest', 'Distance']]
frames = [X_train, X_test]
X = pd.concat(frames)
cat_features = X.columns[X.dtypes == 'object']
df_interact = X.copy()
for i, col1 in enumerate(cat_features):
    for j, col2 in enumerate(cat_features[i + 1:]):
        df_interact[col1 + '_' + col2] = df_interact[col1] + '_' + df_interact[col2]

In [7]:
df_interact.shape

(200000, 23)

In [8]:
from catboost import CatBoostClassifier
X_train_part, X_valid, y_train_part, y_valid = train_test_split(df_interact[:100000], y_train, test_size=0.3, random_state=17)
cat_features_idx = np.where(X_train_part.dtypes == 'object')[0].tolist()

In [25]:
%%time
# cat = CatBoostClassifier(iterations = 500, learning_rate=0.09, loss_function='Logloss', verbose=True, depth=10)
cat = CatBoostClassifier(iterations = 370, learning_rate=1, loss_function='Logloss', verbose=True, depth=10)
cat.fit(df_interact[:100000], y_train, cat_features=cat_features_idx)

0: learn: 0.4585239	total: 2.98s	remaining: 18m 19s
1: learn: 0.4393033	total: 6.78s	remaining: 20m 47s
2: learn: 0.4350898	total: 8.65s	remaining: 17m 37s
3: learn: 0.4331525	total: 11.3s	remaining: 17m 13s
4: learn: 0.4301641	total: 14.5s	remaining: 17m 42s
5: learn: 0.4278765	total: 19.2s	remaining: 19m 22s
6: learn: 0.4271911	total: 20.9s	remaining: 18m 1s
7: learn: 0.4268036	total: 22.1s	remaining: 16m 41s
8: learn: 0.4248887	total: 25.2s	remaining: 16m 51s
9: learn: 0.4231351	total: 28.6s	remaining: 17m 8s
10: learn: 0.4218614	total: 33s	remaining: 17m 55s
11: learn: 0.4195834	total: 37.7s	remaining: 18m 45s
12: learn: 0.4195766	total: 38.3s	remaining: 17m 32s
13: learn: 0.4177413	total: 43.1s	remaining: 18m 14s
14: learn: 0.4161276	total: 48.3s	remaining: 19m 2s
15: learn: 0.4150305	total: 53.3s	remaining: 19m 39s
16: learn: 0.4124954	total: 58.2s	remaining: 20m 8s
17: learn: 0.4096789	total: 1m 2s	remaining: 20m 15s
18: learn: 0.4072954	total: 1m 5s	remaining: 20m 13s
19: learn

153: learn: 0.3151402	total: 11m 6s	remaining: 15m 34s
154: learn: 0.3144836	total: 11m 11s	remaining: 15m 31s
155: learn: 0.3144321	total: 11m 16s	remaining: 15m 27s
156: learn: 0.3144277	total: 11m 20s	remaining: 15m 23s
157: learn: 0.3144228	total: 11m 24s	remaining: 15m 18s
158: learn: 0.3138821	total: 11m 29s	remaining: 15m 15s
159: learn: 0.3138794	total: 11m 32s	remaining: 15m 9s
160: learn: 0.3138754	total: 11m 35s	remaining: 15m 3s
161: learn: 0.3137803	total: 11m 39s	remaining: 14m 58s
162: learn: 0.3136917	total: 11m 43s	remaining: 14m 53s
163: learn: 0.31369	total: 11m 48s	remaining: 14m 49s
164: learn: 0.3136849	total: 11m 52s	remaining: 14m 45s
165: learn: 0.3136775	total: 11m 56s	remaining: 14m 40s
166: learn: 0.31109	total: 12m 1s	remaining: 14m 37s
167: learn: 0.3097545	total: 12m 6s	remaining: 14m 33s
168: learn: 0.3088046	total: 12m 12s	remaining: 14m 30s
169: learn: 0.3083442	total: 12m 16s	remaining: 14m 26s
170: learn: 0.3073979	total: 12m 21s	remaining: 14m 22s
1

303: learn: 0.2675539	total: 22m 9s	remaining: 4m 48s
304: learn: 0.2675511	total: 22m 13s	remaining: 4m 44s
305: learn: 0.2675489	total: 22m 17s	remaining: 4m 39s
306: learn: 0.2675015	total: 22m 21s	remaining: 4m 35s
307: learn: 0.2674847	total: 22m 26s	remaining: 4m 30s
308: learn: 0.2674839	total: 22m 29s	remaining: 4m 26s
309: learn: 0.2674633	total: 22m 33s	remaining: 4m 22s
310: learn: 0.2674632	total: 22m 38s	remaining: 4m 17s
311: learn: 0.2674618	total: 22m 41s	remaining: 4m 13s
312: learn: 0.2674582	total: 22m 45s	remaining: 4m 8s
313: learn: 0.2674529	total: 22m 49s	remaining: 4m 4s
314: learn: 0.2674359	total: 22m 53s	remaining: 3m 59s
315: learn: 0.2674353	total: 22m 56s	remaining: 3m 55s
316: learn: 0.267425	total: 23m	remaining: 3m 50s
317: learn: 0.2674244	total: 23m 5s	remaining: 3m 46s
318: learn: 0.2671878	total: 23m 10s	remaining: 3m 42s
319: learn: 0.2670545	total: 23m 14s	remaining: 3m 37s
320: learn: 0.2664956	total: 23m 19s	remaining: 3m 33s
321: learn: 0.26581

In [26]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_train[70000:], cat.predict(df_interact[70000:100000])), "CAT")
cat_valid_pred = cat.predict_proba(X_valid)[:, 1]
print(roc_auc_score(y_valid, cat_valid_pred))
cat_test_pred = cat.predict_proba(df_interact[100000:])[:, 1]
pd.Series(cat_test_pred, name='dep_delayed_15min').to_csv('cat_2feat.csv', index_label='id', header=True)

0.850966666667 CAT
0.763597958113


In [10]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_train[70000:], cat.predict(df_interact[70000:100000])), "CAT")
cat_valid_pred = cat.predict_proba(X_valid)[:, 1]
print(roc_auc_score(y_valid, cat_valid_pred))

0.871133333333 CAT
0.896627821681


In [11]:
cat_test_pred = cat.predict_proba(df_interact[100000:])[:, 1]
pd.Series(cat_test_pred, name='dep_delayed_15min').to_csv('cat_2feat.csv', index_label='id', header=True)

In [14]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_train[70000:], cat.predict(df_interact[70000:100000])), "CAT")
cat_valid_pred = cat.predict_proba(X_valid)[:, 1]
print(roc_auc_score(y_valid, cat_valid_pred))

0.870766666667 CAT
0.888844459444


In [12]:
print()

In [11]:
train['Root'] = train['Origin'] + "_" + train['Dest']
test['Root'] = test['Origin'] + "_" + test['Dest']

In [12]:
train.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance,dep_delayed_15min,Root
0,c-8,c-21,c-7,1934,AA,ATL,DFW,732,N,ATL_DFW
1,c-4,c-20,c-3,1548,US,PIT,MCO,834,N,PIT_MCO
2,c-9,c-2,c-5,1422,XE,RDU,CLE,416,N,RDU_CLE
3,c-11,c-25,c-6,1015,OO,DEN,MEM,872,N,DEN_MEM
4,c-10,c-7,c-6,1828,WN,MDW,OMA,423,Y,MDW_OMA


In [13]:
X_train, y_train = train[['Month', 'DayofMonth', 'DayOfWeek', 'DepTime', 'UniqueCarrier','Distance', 'Root']], train['dep_delayed_15min'].map({'Y': 1, 'N': 0})
X_test = test[['Month', 'DayofMonth', 'DayOfWeek', 'DepTime', 'UniqueCarrier', 'Distance', 'Root']]
frames = [X_train, X_test]
X = pd.concat(frames)
df_interact = X.copy()

In [14]:
df_interact.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Distance,Root
0,c-8,c-21,c-7,1934,AA,732,ATL_DFW
1,c-4,c-20,c-3,1548,US,834,PIT_MCO
2,c-9,c-2,c-5,1422,XE,416,RDU_CLE
3,c-11,c-25,c-6,1015,OO,872,DEN_MEM
4,c-10,c-7,c-6,1828,WN,423,MDW_OMA


In [15]:
%%time
df_interact = pd.get_dummies(df_interact, columns=['Month', 'DayofMonth', 'DayOfWeek', 'UniqueCarrier', 'Root'])

Wall time: 21.7 s


In [16]:
df_interact.head()

,DepTime,Distance,Month_c-1,Month_c-10,Month_c-11,Month_c-12,Month_c-2,Month_c-3,Month_c-4,Month_c-5,...,Root_XNA_MSP,Root_XNA_ORD,Root_XNA_SLC,Root_YAK_CDV,Root_YAK_JNU,Root_YKM_SLC,Root_YUM_IPL,Root_YUM_LAS,Root_YUM_LAX,Root_YUM_PHX
0,1934,732,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1548,834,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,1422,416,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1015,872,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1828,423,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
%%time
from sklearn.linear_model import LogisticRegression
logit = LogisticRegression()

logit.fit(df_interact[:100000], y_train)
logit_test_pred = logit.predict_proba(df_interact[100000:])[:, 1]

pd.Series(logit_test_pred, name='dep_delayed_15min').to_csv('logit_2feat.csv', index_label='id', header=True)

Wall time: 12min 2s


In [17]:
XGBClassifier?

In [ ]:
%%time
xgb = XGBClassifier(n_estimators=100)
xgb.fit(df_interact[:100000], y_train[:100000])

In [ ]:
print(accuracy_score(y_train[10000:20000], xgb.predict(df_interact[10000:20000])))
xgb_valid_pred = xgb.predict_proba(X_valid)[:, 1]
print(roc_auc_score(y_valid, xgb_valid_pred))

In [15]:
xgb_test_pred = xgb.predict_proba(df_interact[100000:])[:, 1]
pd.Series(cat_test_pred, name='dep_delayed_15min').to_csv('cat_2feat.csv', index_label='id', header=True)

In [10]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_train[70000:], cat.predict(df_interact[70000:100000])), "CAT")
cat_valid_pred = cat.predict_proba(X_valid)[:, 1]
print(roc_auc_score(y_valid, cat_valid_pred))

0.868466666667 CAT
0.88615507903


In [23]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_train[70000:], cat.predict(df_interact[70000:100000])), "CAT")
cat_valid_pred = cat.predict_proba(X_valid)[:, 1]
print(roc_auc_score(y_valid, cat_valid_pred))

0.8663 CAT
0.88672593548


In [ ]:
%%time
classifier = XGBClassifier(n_jobs = -1)
parameters_grid = {
    'learning_rate' : np.linspace(0.01,0.2,5),
    'min_child_weight' : list(range(1,10,3)),
    'max_depth': list(range(1,10,3))
}
cv = cross_validation.StratifiedShuffleSplit(test_size = 0.3, random_state=100)
grid_cv = grid_search.GridSearchCV(classifier, parameters_grid, scoring = 'accuracy', cv = cv)

grid_cv.fit(df_interact[:100000], y_train)
print(grid_cv.best_score_)
print(grid_cv.best_params_)

In [17]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_train[70000:], cat.predict(df_interact[70000:100000])), "CAT")
cat_valid_pred = cat.predict_proba(X_valid)[:, 1]
print(roc_auc_score(y_valid, cat_valid_pred))

0.817066666667 CAT
0.817605735331


In [11]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_train[70000:], cat.predict(df_interact[70000:100000])), "CAT")
cat_valid_pred = cat.predict_proba(X_valid)[:, 1]
print(roc_auc_score(y_valid, cat_valid_pred))

0.8186 CAT
0.815251613484


In [18]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_train[70000:], cat.predict(df_interact[70000:100000])), "CAT")
cat_valid_pred = cat.predict_proba(X_valid)[:, 1]
print(roc_auc_score(y_valid, cat_valid_pred))

0.808666666667 CAT
0.788567082996


In [12]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_train[70000:], cat.predict(df_interact[70000:100000])), "CAT")
cat_valid_pred = cat.predict_proba(X_valid)[:, 1]
print(roc_auc_score(y_valid, cat_valid_pred))

0.8672 CAT
0.878746334764


In [19]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_train[70000:], cat.predict(df_interact[70000:100000])), "CAT")
cat_valid_pred = cat.predict_proba(X_valid)[:, 1]
print(roc_auc_score(y_valid, cat_valid_pred))

0.858566666667 CAT
0.820487509682


In [ ]:
def evaluate_cross_validation_1(clf, X, y, K):
    K=5
    estimator = clf
    cv_sk = model_selection.StratifiedKFold(K, shuffle=True, random_state=135)
    scores = model_selection.cross_val_score(estimator, X, y, scoring='accuracy', cv=cv_sk) 
    print(scores)
    print("Mean score: {0:.3f} (+/-{1:.3f})".format(scores.mean(), scores.std()))

In [ ]:
evalime
from sklearn import model_selection
evaluate_cross_validation()

In [6]:
logit = LogisticRegression()

logit.fit(X_train_part, y_train_part)
logit_valid_pred = logit.predict_proba(X_valid)[:, 1]

roc_auc_score(y_valid, logit_valid_pred)

0.67956914653526068

In [7]:
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.fit_transform(X_test)

logit.fit(X_train_scaled, y_train)
logit_test_pred = logit.predict_proba(X_test_scaled)[:, 1]

pd.Series(logit_test_pred, name='dep_delayed_15min').to_csv('logit_2feat.csv', index_label='id', header=True)

C:\Users\Qoffee\Anaconda3v2\lib\site-packages\sklearn\utils\validation.py:444: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


Второй бенчмарк, представленный в рейтинге соревнования, был получен так:
- Признаки `Distance` и  `DepTime` брались без изменений
- Создан признак "маршрут" из исходных `Origin` и `Dest`
- К признакам `Month`, `DayofMonth`, `DayOfWeek`, `UniqueCarrier` и "маршрут" применено OHE-преобразование (`LabelBinarizer`)
- Выделена отложенная выборка
- Обучалась логистическая регрессия и градиентный бустинг (xgboost), гиперпараметры бустинга настраивались на кросс-валидации, сначала те, что отвечают за сложность модели, затем число деревьев фиксировалось равным 500 и настраивался шаг градиентного спуска
- С помощью `cross_val_predict` делались прогнозы обеих моделей на кросс-валидации (именно предсказанные вероятности), настраивалась линейная смесь ответов логистической регрессии и градиентного бустинга вида $w_1 * p_{logit} + (1 - w_1) * p_{xgb}$, где $p_{logit}$ – предсказанные логистической регрессией вероятности класса 1, $p_{xgb}$ – аналогично. Вес $w_1$ подбирался вручную. 
- В качестве ответа для тестовой выборки бралась аналогичная комбинация ответов двух моделей, но уже обученных на всей обучающей выборке.

Описанный план ни к чему не обязывает – это просто то, как решение получил автор задания. Возможно, вы не захотите следовать намеченному плану, а добавите, скажем, пару хороших признаков и обучите лес из тысячи деревьев.

Удачи!

In [10]:
X.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance
0,c-8,c-21,c-7,1934,AA,ATL,DFW,732
1,c-4,c-20,c-3,1548,US,PIT,MCO,834
2,c-9,c-2,c-5,1422,XE,RDU,CLE,416
3,c-11,c-25,c-6,1015,OO,DEN,MEM,872
4,c-10,c-7,c-6,1828,WN,MDW,OMA,423


In [23]:
df_interact.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance,Month_DayofMonth,Month_DayOfWeek,DayofMonth_DayOfWeek
0,c-8,c-21,c-7,1934,AA,ATL,DFW,732,c-8_c-21,c-8_c-7,c-21_c-7
1,c-4,c-20,c-3,1548,US,PIT,MCO,834,c-4_c-20,c-4_c-3,c-20_c-3
2,c-9,c-2,c-5,1422,XE,RDU,CLE,416,c-9_c-2,c-9_c-5,c-2_c-5
3,c-11,c-25,c-6,1015,OO,DEN,MEM,872,c-11_c-25,c-11_c-6,c-25_c-6
4,c-10,c-7,c-6,1828,WN,MDW,OMA,423,c-10_c-7,c-10_c-6,c-7_c-6


In [24]:
df_interact.columns

Index(['Month', 'DayofMonth', 'DayOfWeek', 'DepTime', 'UniqueCarrier',
       'Origin', 'Dest', 'Distance', 'Month_DayofMonth', 'Month_DayOfWeek',
       'DayofMonth_DayOfWeek'],
      dtype='object')

In [28]:
%%time
df_interact = pd.get_dummies(df_interact, columns=df_interact.columns[df_interact.dtypes == 'object'])

Wall time: 2.89 s


In [29]:
df_interact.shape

(200000, 1355)

In [31]:
X_train, y_train = df_interact[:100000].values, y_train
X_test = df_interact[100000:].values

logit.fit(X_train, y_train)
logit_test_pred = logit.predict_proba(X_test)[:, 1]

pd.Series(logit_test_pred, name='dep_delayed_15min').to_csv('logit_2feat.csv', index_label='id', header=True)

In [82]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import BaggingClassifier 
bc = BaggingClassifier()
bc.fit(X_train, y_train)
accuracy_score(y_train, bc.predict(X_train))
bc_test_pred = rfc.predict_proba(X_test)[:, 1]
pd.Series(bc_test_pred, name='dep_delayed_15min').to_csv('bc_2feat.csv', index_label='id', header=True)

In [81]:
from sklearn.ensemble import GradientBoostingClassifier 
gbc = GradientBoostingClassifier()
gbc.fit(X_train, y_train)
accuracy_score(y_train, gbc.predict(X_train))
gbc_test_pred = gbc.predict_proba(X_test)[:, 1]
pd.Series(gbc_test_pred, name='dep_delayed_15min').to_csv('gbc_2feat.csv', index_label='id', header=True)

In [83]:
accuracy_score(y_train, gbc.predict(X_train))

0.81808999999999998

In [61]:
from sklearn.calibration import CalibratedClassifierCV
cc = CalibratedClassifierCV()
cc.fit(X_train[:70000], y_train[:70000])
accuracy_score(y_train[70000:], cc.predict(X_train[70000:]))

0.80840000000000001

In [62]:
from sklearn.dummy import DummyClassifier
dc = DummyClassifier()
dc.fit(X_train[:70000], y_train[:70000])
accuracy_score(y_train[70000:], dc.predict(X_train[70000:]))

0.69353333333333333

In [63]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
qda = QuadraticDiscriminantAnalysis()
lda = LinearDiscriminantAnalysis()
qda.fit(X_train[:70000], y_train[:70000])
print(accuracy_score(y_train[70000:], qda.predict(X_train[70000:])), "QDA")
lda.fit(X_train[:70000], y_train[:70000])
print(accuracy_score(y_train[70000:], lda.predict(X_train[70000:])), "LDA")

C:\Users\Qoffee\Anaconda3v2\lib\site-packages\sklearn\discriminant_analysis.py:682: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


0.209333333333 QDA


C:\Users\Qoffee\Anaconda3v2\lib\site-packages\sklearn\discriminant_analysis.py:388: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")


0.810833333333 LDA


In [64]:
from sklearn.linear_model import RidgeClassifierCV
rcc = RidgeClassifierCV()
rcc.fit(X_train[:70000], y_train[:70000])
print(accuracy_score(y_train[70000:], rcc.predict(X_train[70000:])), "GPC")

0.8091 GPC


In [74]:
from sklearn.linear_model import LogisticRegression
log = LogisticRegression()
log.fit(X_train[:70000], y_train[:70000])
print(accuracy_score(y_train[70000:], log.predict(X_train[70000:])), "GPC")

0.812133333333 GPC


In [65]:
from sklearn.linear_model import PassiveAggressiveClassifier
pac = PassiveAggressiveClassifier()
pac.fit(X_train[:70000], y_train[:70000])
print(accuracy_score(y_train[70000:], pac.predict(X_train[70000:])), "GPC")

C:\Users\Qoffee\Anaconda3v2\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:84: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.passive_aggressive.PassiveAggressiveClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


0.8084 GPC


In [66]:
from sklearn.naive_bayes import BernoulliNB
bn = BernoulliNB()
bn.fit(X_train[:70000], y_train[:70000])
print(accuracy_score(y_train[70000:], bn.predict(X_train[70000:])), "BN")

0.806066666667 BN


In [67]:
from sklearn.neighbors import KNeighborsClassifier
kc = KNeighborsClassifier()
kc.fit(X_train[:70000], y_train[:70000])
print(accuracy_score(y_train[70000:], kc.predict(X_train[70000:])), "KC")

0.791733333333 KC


In [68]:
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import Perceptron
mlp = MLPClassifier()
per = Perceptron()
mlp.fit(X_train[:70000], y_train[:70000])
print(accuracy_score(y_train[70000:], mlp.predict(X_train[70000:])), "MLP")
per.fit(X_train[:70000], y_train[:70000])
print(accuracy_score(y_train[70000:], per.predict(X_train[70000:])), "PER")

C:\Users\Qoffee\Anaconda3v2\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:84: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.perceptron.Perceptron'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


0.805266666667 MLP
0.8084 PER


In [69]:
rfc = XGBClassifier(max_depth=3)
rfc.fit(X_train[:70000], y_train[:70000])
accuracy_score(y_train[70000:], rfc.predict(X_train[70000:]))

0.81496666666666662

In [71]:
%%time
df_interact = pd.get_dummies(X, columns=X.columns[X.dtypes == 'object'])
X_train, y_train = df_interact[:100000].values, y_train
X_test = df_interact[100000:].values

Wall time: 2.02 s


In [80]:
%%time
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import Perceptron
mlp = MLPClassifier()
per = Perceptron()
mlp.fit(X_train, y_train)
print(accuracy_score(y_train, mlp.predict(X_train)), "MLP")
mlp_test_pred = mlp.predict_proba(X_test)[:, 1]
pd.Series(mlp_test_pred, name='dep_delayed_15min').to_csv('mlp_2feat.csv', index_label='id', header=True)
per.fit(X_train, y_train)
print(accuracy_score(y_train, per.predict(X_train)), "PER")

C:\Users\Qoffee\Anaconda3v2\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:84: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.perceptron.Perceptron'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


0.68621 MLP
0.80956 PER
Wall time: 50.8 s


In [78]:
%%time
from sklearn.ensemble import AdaBoostClassifier 
from sklearn.ensemble import ExtraTreesClassifier 
abc = AdaBoostClassifier()
etc = ExtraTreesClassifier()
abc.fit(X_train, y_train)
print(accuracy_score(y_train, abc.predict(X_train)), "ABC")
abc_test_pred = abc.predict_proba(X_test)[:, 1]
pd.Series(abc_test_pred, name='dep_delayed_15min').to_csv('abc_2feat.csv', index_label='id', header=True)

0.81311 ABC
Wall time: 36.5 s


In [ ]:
from sklearn import svm, grid_search
def svc_param_selection(X, y, nfolds):
    Cs = [0.001, 0.01, 0.1, 1, 10]
    gammas = [0.001, 0.01, 0.1, 1]
    param_grid = {'C': Cs, 'gamma' : gammas}
    grid_search = GridSearchCV(svm.SVC(kernel='rbf'), param_grid, cv=nfolds)
    grid_search.fit(X, y)
    grid_search.best_params_
    return grid_search.best_params_

In [84]:
def evaluate_cross_validation_1(clf, X, y, K):
    K=5
    estimator = clf
    cv_sk = model_selection.StratifiedKFold(K, shuffle=True, random_state=135)
    scores = model_selection.cross_val_score(estimator, X, y, scoring='accuracy', cv=cv_sk) 
    print(scores)
    print("Mean score: {0:.3f} (+/-{1:.3f})".format(scores.mean(), scores.std()))
    
    
def evaluate_cross_validation_2(clf, X, y, K):
    estimator = pipeline.Pipeline(steps = [       
    ('feature_processing', pipeline.FeatureUnion(transformer_list = [        
            #binary
            ('binary_variables_processing', preprocessing.FunctionTransformer(lambda X: X[:, binary_data_indices])), 
                    
            #numeric
            ('numeric_variables_processing', pipeline.Pipeline(steps = [
                ('selecting', preprocessing.FunctionTransformer(lambda X: X[:, numeric_data_indices])),
                ('scaling', preprocessing.StandardScaler(with_mean = 0, with_std = 1))            
                        ])),
        
            #categorical
            ('categorical_variables_processing', pipeline.Pipeline(steps = [
                ('selecting', preprocessing.FunctionTransformer(lambda X: X[:, categorical_data_indices])),
                ('hot_encoding', preprocessing.OneHotEncoder(handle_unknown = 'ignore'))            
                        ])),
        ])),
    ('model_fitting', clf)
    ]
    )
    cv_sk = model_selection.StratifiedKFold(K, shuffle=True, random_state=135)
    scores = model_selection.cross_val_score(estimator, X, y, scoring='accuracy', cv=cv_sk) 
    print(scores)
    print("Mean score: {0:.3f} (+/-{1:.3f})".format(scores.mean(), scores.std()))

In [ ]:
%%time
from sklearn.ensemble import ExtraTreesClassifier
from sklearn import model_selection
classifier = ExtraTreesClassifier(n_estimators = 30, n_jobs=-1, random_state=100)
parameters_grid = {
    'criterion': ['gini', 'entropy'],
    'max_features' : ['sqrt','log2',None],
    'max_depth': list(range(3,16,3)),
    'min_samples_split' : list(range(2,12,3)),
    'min_samples_leaf': list(range(1,10,3)),
}
cv = model_selection.StratifiedShuffleSplit(test_size = 0.3, random_state=100)
grid_cv = model_selection.GridSearchCV(classifier, parameters_grid, scoring = 'accuracy', cv = cv)

grid_cv.fit(X_train, y_train)
print(grid_cv.best_score_)
print(grid_cv.best_params_)

In [ ]:
%%time
from sklearn.ensemble import GradientBoostingClassifier
classifier = GradientBoostingClassifier(n_estimators = 200, random_state=100)
parameters_grid = {
    'max_features' : ['sqrt','log2'],
    'loss' : ['deviance', 'exponential'],
#     'min_samples_split' : list(range(2,9,3)),
#     'min_samples_leaf': list(range(1,11,3)),
    'max_depth': list(range(3,16,3)),
    'max_features': list(range(1,11,2))
}
cv = model_selection.StratifiedShuffleSplit(test_size = 0.3, random_state=100)
grid_cv = model_selection.GridSearchCV(classifier, parameters_grid, scoring = 'accuracy', cv = cv)

grid_cv.fit(X, y_c)
print(grid_cv.best_score_)
print(grid_cv.best_params_)

In [ ]:
%%time
classifier = XGBClassifier()
parameters_grid = {
    'learning_rate' : np.linspace(0.01,0.3,5),
    'min_child_weight' : list(range(1,10,3)),
    'max_depth': list(range(1,10,3))
}
cv = cross_validation.StratifiedShuffleSplit(test_size = 0.3, random_state=100)
grid_cv = grid_search.GridSearchCV(classifier, parameters_grid, scoring = 'accuracy', cv = cv)

grid_cv.fit(X, y_c)
print(grid_cv.best_score_)
print(grid_cv.best_params_)